In [1]:
import pandas as pd

In [2]:
dataset = pd.read_csv('dataset\Twitter_Emotion_Dataset.csv')

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4401 entries, 0 to 4400
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   4401 non-null   object
 1   tweet   4401 non-null   object
dtypes: object(2)
memory usage: 68.9+ KB


In [4]:
texts = dataset['tweet'].tolist()

In [5]:
from text_preprocessing.asycro_text_processor import TextProcessor  

BATCH_SIZE = 1000

async def process_batch(batch):
    processor = TextProcessor()
    return await processor.process_texts(batch)

async def process_large_dataset(texts):
    results = []
    for i in range(0, len(texts), BATCH_SIZE):
        batch = texts[i:i + BATCH_SIZE]
        batch_results = await process_batch(batch)
        results.extend(batch_results)
    return results

d:\Funda NLP\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#cleaned_texts = asyncio.run(process_large_dataset(texts))
cleaned_texts = await process_large_dataset(texts)

# Simpan hasil ke dataframe
dataset['cleaned_text'] = cleaned_texts

In [7]:
dataset['cleaned_text'][1]

'cewek lho kayak rasain sibuk jaga rasain sakit haid panik pulang malam orang asing wajar korban takut curhat bela hujat'

In [7]:
dataset

,label,tweet,cleaned_text
0,anger,"Soal jln Jatibaru,polisi tdk bs GERTAK gubernu...",jalan jatibaru polisi geak gubernur emangny po...
1,anger,"Sesama cewe lho (kayaknya), harusnya bisa lebi...",cewek lho kayak rasain sibuk jaga rasain sakit...
2,happy,Kepingin gudeg mbarek Bu hj. Amad Foto dari go...,gudeg mbarek hj amad foto google sengaja biar ...
3,anger,"Jln Jatibaru,bagian dari wilayah Tn Abang.Peng...",jalan jatibaru wilayah tn abang atur wilayah t...
4,happy,"Sharing pengalaman aja, kemarin jam 18.00 bata...",sharing alam kemarin jam batalin tiket stasiun...
...,...,...,...
4396,love,"Tahukah kamu, bahwa saat itu papa memejamkan m...",papa mejam mata tahan gejolak batin papa tapu ...
4397,fear,Sulitnya menetapkan Calon Wapresnya Jokowi di ...,sulit calon wapresnya jokowi pilpres salah gem...
4398,anger,"5. masa depannya nggak jelas. lha iya, gimana ...",lha coba lulus seni nari kerja nari nari hasil...
4399,happy,[USERNAME] dulu beneran ada mahasiswa Teknik U...,name beneran mahasiswa teknik ui nyata cinta p...


## FAST TEXT

In [ ]:
!pip install keras
!pip install tensorflow

In [12]:
from urllib.request import urlopen
import gzip
import numpy as np
from keras.layers import TextVectorization
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [16]:
# Membuat embedding matrix
def create_embedding_matrix(vocab_and_vectors, word_index, embedding_dim):
    """
    Membuat matriks embedding berdasarkan model embedding dan indeks kata.

    Args:
        vocab_and_vectors (dict): Peta kata ke vektor embedding dari model FastText.
        word_index (dict): Peta kata ke indeks kata dari hasil tokenisasi.
        embedding_dim (int): Dimensi vektor embedding.

    Returns:
        np.ndarray: Matriks embedding dengan bentuk (jumlah_kata, dimensi_embedding).
    """
    num_words = len(word_index) + 1  # Jumlah kata termasuk indeks 0
    embedding_matrix = np.zeros((num_words, embedding_dim))  # Inisialisasi matriks dengan 0

    for word, i in word_index.items():
        if i >= num_words:
            continue  # Lewati jika indeks lebih dari jumlah kata
        vector = vocab_and_vectors.get(word)  # Ambil vektor kata
        if vector is not None:
            embedding_matrix[i] = vector  # Isi matriks embedding dengan vektor kata
    return embedding_matrix

# Dimensi embedding FastText
embedding_dim = 300

# Tokenize and pad the text data
max_vocab_size = 10000  # Jumlah maksimal kata dalam vocab
max_seq_length = 400    # Panjang maksimum urutan

# Inisialisasi TextVectorization
vectorizer = TextVectorization(
    max_tokens=max_vocab_size,
    output_sequence_length=max_seq_length
)

# Fit TextVectorization pada data teks
text_data = dataset['cleaned_text']
vectorizer.adapt(text_data)


# Mendownload model FastText untuk bahasa Indonesia
url = 'https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz'

with gzip.open(urlopen(url), 'rt', encoding='utf-8') as file:
    vocab_and_vectors = {}  # memetakan kata ke vektor
    for line in file:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], dtype='float32')
        vocab_and_vectors[word] = vector

# Mendapatkan daftar vocabulary dari TextVectorization
vocabulary = vectorizer.get_vocabulary()

# Membuat word_index (dictionary kata ke indeks)
word_index = {word: idx for idx, word in enumerate(vocabulary)}

# Membuat embedding matrix
embedding_matrix = create_embedding_matrix(vocab_and_vectors, word_index, embedding_dim)

In [17]:
# Count unique words
num_unique_words = len(vectorizer.get_vocabulary())
print("Number of unique words:", num_unique_words)

Number of unique words: 10000


In [29]:
# Contoh teks dan sequence indexing
for i, text in enumerate(text_data[:3]):  # Ambil 3 contoh pertama
    indexed_sequence = vectorizer(text).numpy()  # Hasil sequence indexing
    print(f"Original Text {i + 1}:\n{text}")
    print(f"Indexed Sequence {i + 1}:\n{indexed_sequence}")

Original Text 1:
jalan jatibaru polisi geak gubernur emangny polisi pmbhasan politik atur wilayah hak gubernur tn abang turun turun pelik sabar name name url
Indexed Sequence 1:
[  19 4888  482    1  473    1  482 7712  186  372 1418  407  473 3791
  412  264  264 7887  150    2    2    4    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0 

In [30]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Tokenisasi dan padding teks
sequences = [vectorizer(text).numpy() for text in text_data[:3]]
X_padded = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

for i, padded_seq in enumerate(X_padded):
    print(f"\nPadded Sequence {i + 1}:\n{padded_seq}")


Padded Sequence 1:
[  19 4888  482    1  473    1  482 7712  186  372 1418  407  473 3791
  412  264  264 7887  150    2    2    4    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0  

In [31]:
# Contoh embedding dari kata dalam vocabulary
example_words = list(word_index.keys())[:3]  # Ambil 3 kata pertama dari word_index

for word in example_words:
    example_embedding = vocab_and_vectors.get(word)  # Ambil embedding dari FastText
    print(f"\nExample Word Embedding for '{word}':\n{example_embedding}")


Example Word Embedding for '[UNK]':
None

Example Word Embedding for 'name':
[-4.790e-02  1.000e-02 -1.570e-02 -1.430e-02 -1.513e-01 -6.690e-02
 -2.250e-02  3.040e-02  6.500e-03 -9.960e-02 -4.380e-02  1.064e-01
 -9.720e-02  1.960e-02  1.995e-01  2.760e-02  1.250e-01 -5.600e-03
  5.900e-03  1.171e-01 -2.657e-01  1.440e-02 -1.500e-02  2.545e-01
  7.250e-02 -2.690e-02  1.800e-03  2.870e-02  5.480e-02  1.357e-01
  3.789e-01 -1.700e-03  1.781e-01  1.300e-01 -5.900e-02  8.300e-02
 -1.071e-01 -4.400e-02  2.202e-01 -5.480e-02  1.088e-01  1.042e-01
 -6.490e-02 -8.110e-02 -7.120e-02 -2.000e-04  1.467e-01  1.520e-02
  9.740e-02 -2.310e-02 -1.160e-02  1.340e-01 -1.491e-01 -5.080e-02
  1.472e-01  1.311e-01 -4.210e-02  2.110e-02  1.080e-01 -6.340e-02
  6.160e-02 -7.160e-02 -1.300e-02  5.080e-02  2.351e-01  3.580e-02
 -4.950e-02  1.411e-01  2.060e-01  8.100e-03  4.830e-02 -4.730e-02
  7.500e-02 -9.390e-02 -4.080e-02  9.160e-02 -1.749e-01 -1.360e-02
 -1.465e-01  3.000e-04 -1.250e-02 -8.030e-02  6.850

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def find_similar_words(word, embedding_matrix, word_index, top_n=5):
    """
    Temukan kata-kata yang mirip berdasarkan embedding matrix.
    
    Args:
        word (str): Kata yang ingin dicari kemiripannya.
        embedding_matrix (np.ndarray): Matriks embedding.
        word_index (dict): Peta kata ke indeks kata.
        top_n (int): Jumlah kata mirip yang ingin diambil.
    
    Returns:
        list of tuple: Daftar kata mirip dan skor kemiripannya.
    """
    if word in word_index:
        idx = word_index[word]
        embedding = embedding_matrix[idx].reshape(1, -1)  # Ubah ke array 2D untuk cosine similarity
        cosine_similarities = cosine_similarity(embedding, embedding_matrix)
        similar_words = [(word, 1.0)]  # Tambahkan kata input dengan similarity 1.0
        for idx_similar in cosine_similarities.argsort()[0][-top_n-1:-1][::-1]:
            similar_word = next(key for key, value in word_index.items() if value == idx_similar)
            similarity = cosine_similarities[0][idx_similar]
            similar_words.append((similar_word, similarity))
        return similar_words
    else:
        print(f"Word '{word}' not found in vocabulary.")
        return []

# Contoh penggunaan: Cari kata mirip dengan "jalan"
similar_words = find_similar_words("jalan", embedding_matrix, word_index, top_n=5)

print("\nSimilar Words to 'jalan':")
for word, similarity in similar_words:
    print(f"{word}: {similarity:.4f}")


Similar Words to 'jalan':
jalan: 1.0000
jalur: 0.6760
setapak: 0.6255
tol: 0.6167
raya: 0.6082
pinggir: 0.5978


In [ ]:
vocabulary = vectorizer.get_vocabulary()
example_word = list(word_index.keys())[0]
print(vocabulary)

<bound method TextVectorization.get_vocabulary of <TextVectorization name=text_vectorization_3, built=True>>
